This notebook is a supplement to the paper ["The Orthologic of Epistemic Modals"](https://escholarship.org/uc/item/0ss5z8g3) by [Wesley H. Holliday](mailto:wesholliday@berkeley.edu) and [Matthew Mandelkern](mandelkern@nyu.edu).

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s [interface](https://www.nltk.org/howto/inference.html) to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations. For example, we treat the logical principle $\Box\varphi \vdash \varphi$, corresponding to the lattice inequality $\Box a\leq a$, as the equation $\Box a = \Box a\wedge a$.

## Outline

**1. [Lattice axioms](#1)**

**2. [Bounded lattice axioms](#2)**

**3. [Ortholattice axioms](#3)**

**4. [Boolean subalgebra axioms](#4)**

**5. [Modal axioms](#5)**

**6. [Epistemic axioms](#6)**

**7. [Conditional axioms](#7)**

**8. [Independence of axioms](#8)**

**9. [The conditional epistemic ortholattice from Figure 12](#9)**

**10. [Example proofs](#10)**

**11. [Avoiding collapse](#11)**

**12. [Modalized Import-Export](#12)**

**13. [Qualified Collapse](#13)**

**14. [Provable principles for which Prover9 does not find a proof](#14)**

**15. [A more economical axiomatization without $\vee$, $\bot$, or $\Diamond$](#14)**

In [1]:
from nltk.test.inference_fixt import setup_module 

setup_module()

from nltk import *
from nltk.sem.drt import DrtParser
from nltk.sem import logic
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## 1. Lattice axioms<a id='1'></a>

In [2]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

In [3]:
#By including a definition of the covering relation,
#one can quickly read off Hasse diagrams of lattices from the Mace4 output.
cover_def = read_expr('covered_by(x,y) <-> (-(x=y) & x = And(x,y) & -exists z.(-(z=x) & -(z=y) & x = And(x,z) & z = And(z,y)))')

## 2. Bounded lattice axioms<a id='2'></a>

In [4]:
bot = read_expr('Or(x,Bot) = x')
top = read_expr('And(x,Top) = x')

bounded_lattice = lattice + [bot,top]

## 3. Ortholattice axioms<a id='3'></a>

In [5]:
lem = read_expr('Or(x,Not(x)) = Top')
contra = read_expr('And(x,Not(x)) = Bot')

invol = read_expr('Not(Not(x)) = x')

de_morgan = read_expr('Not(And(x,y)) = Or(Not(x),Not(y))')

ortho_lattice = bounded_lattice + [lem,contra,invol,de_morgan]

In [6]:
#We do not want distributivity, but we consider it below
dist = read_expr('And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

In [7]:
#Check that the De Morgan law dual to de_morgan above follows
goal = read_expr('Not(Or(x,y)) = And(Not(x),Not(y))')
prover = Prover9Command(goal, assumptions = ortho_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42554 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 70.


1 Not(Or(x,y)) = And(Not(x),Not(y)).  [goal].
16 Not(Not(x)) = x.  [assumption].
17 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption].
18 Not(Or(c1,c2)) != And(Not(c1),Not(c2)).  [deny(1)].
47 Not(Or(Not(x),Not(y))) = And(x,y).  [para(17(a,1),16(a,1,1))].
169 Not(Or(x,Not(y))) = And(Not(x),y).  [para(16(a,1),47(a,1,1,1))].
278 Not(Or(x,y)) = And(Not(

In [8]:
#Check that the distributive law does not follow from ortholattice axioms
goal = dist
mb = MaceCommand(goal, assumptions = ortho_lattice + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 2.

c2 = 3.

c3 = 4.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 0.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 0.
And(5,4) = 0.
And(5,5) = 5.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(0,4) = 4.
Or(0,5) = 5.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(1,4) = 1.
Or(1,5) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(2,4) = 1.
Or(2,5) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.
Or(3,4) = 1.
Or(3,5) = 1.
Or(4,0)

In [9]:
#Check that adding the pseudocomplementation principle allows the derivability of distributivity
pseudo = read_expr('And(x,y) = Bot -> y = And(y,Not(x))')
goal = dist
prover = Prover9Command(goal, assumptions = ortho_lattice + [pseudo])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42558 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.39 (+ 0.01) seconds.
% Length of proof is 93.
% Level of proof is 20.
% Maximum clause weight is 22.000.
% Given clauses 309.


1 And(x,y) = Bot -> y = And(y,Not(x)).  [assumption].
2 And(x,Or(y,z)) = Or(And(x,y),And(x,z)).  [goal].
3 Or(x,x) = x.  [assumption].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
7 Or(x,Or(y,z)) = Or(Or(x,y),z).  [assumption].
8 Or(Or(x,y),z) = Or(x,Or(y,z)).  [copy(7),flip(a)].
9 And(x,And(y,z)) = And(And(x,y),z).  

In [10]:
#Conversely, adding distributivity allows the derivation of the pseudocomplementation principle
goal = pseudo
prover = Prover9Command(goal, assumptions = ortho_lattice + [dist])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42561 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 20.
% Level of proof is 6.
% Maximum clause weight is 13.000.
% Given clauses 50.


1 And(x,y) = Bot -> y = And(y,Not(x)).  [goal].
4 Or(x,y) = Or(y,x).  [assumption].
5 And(x,y) = And(y,x).  [assumption].
11 And(x,Or(x,y)) = x.  [assumption].
12 Or(x,Bot) = x.  [assumption].
13 And(x,Top) = x.  [assumption].
14 Or(x,Not(x)) = Top.  [assumption].
16 Not(Not(x)) = x.  [assumption].
17 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption

## 4. Boolean subalgebra axioms<a id='4'></a>

In [11]:
B_top = read_expr('B(Top)')
B_not = read_expr('B(x) -> B(Not(x))')
B_and = read_expr('(B(x) & B(y)) -> B(And(x,y))')
B_or = read_expr('(B(x) & B(y)) -> B(Or(x,y))')
B_dist = read_expr('((B(x) & B(y)) & B(z)) -> And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

ortho_boolean_lattice = ortho_lattice + [B_top, B_not, B_and, B_or, B_dist]

In [12]:
#Check that the other distributive law dual to B_dist above follows
goal = read_expr('((B(x) & B(y)) & B(z)) -> Or(x,And(y,z)) = And(Or(x,y),Or(x,z))')
prover = Prover9Command(goal, assumptions = ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42563 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.07 (+ 0.01) seconds.
% Length of proof is 50.
% Level of proof is 7.
% Maximum clause weight is 19.000.
% Given clauses 51.


1 B(x) -> B(Not(x)).  [assumption].
2 B(x) & B(y) -> B(And(x,y)).  [assumption].
3 B(x) & B(y) -> B(Or(x,y)).  [assumption].
4 B(x) & B(y) & B(z) -> And(x,Or(y,z)) = Or(And(x,y),And(x,z)).  [assumption].
5 B(x) & B(y) & B(z) -> Or(x,And(y,z)) = And(Or(x,y),Or(x,z)).  [goal].
6 Or(x,x) = x.  [assumption].
7 And(x,x) = x.  [assumption].
8 Or(x,y) = O

## 5. Modal axioms<a id='5'></a>

In [13]:
box_and = read_expr('Box(And(x,y)) = And(Box(x),Box(y))')
box_top = read_expr('Box(Top) = Top')
diamond_dual = read_expr("Diamond(x) = Not(Box(Not(x)))")

modal_ortho_boolean_lattice = ortho_boolean_lattice + [box_and, box_top, diamond_dual]

In [14]:
#Check that Diamond distributes over disjunction
goal = read_expr('Diamond(Or(x,y)) = Or(Diamond(x),Diamond(y))')
prover = Prover9Command(goal, assumptions = modal_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42565 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.02 (+ 0.00) seconds.
% Length of proof is 11.
% Level of proof is 4.
% Maximum clause weight is 16.000.
% Given clauses 107.


5 Diamond(Or(x,y)) = Or(Diamond(x),Diamond(y)).  [goal].
20 Not(Not(x)) = x.  [assumption].
21 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption].
28 Box(And(x,y)) = And(Box(x),Box(y)).  [assumption].
30 Diamond(x) = Not(Box(Not(x))).  [assumption].
31 Diamond(Or(c1,c2)) != Or(Diamond(c1),Diamond(c2)).  [deny(5)].
32 Not(Box(Not(Or(c1,c2)))) != Or(N

## 6. Epistemic axioms<a id='6'></a>

In [15]:
factive = read_expr('Box(x) = And(Box(x),x)')
episteme = read_expr('And(Not(x),Diamond(x)) = Bot')

epistemic_ortho_boolean_lattice = modal_ortho_boolean_lattice + [factive, episteme]

In [16]:
#Check that another form of Wittgenstein sentence is contradictory
goal = read_expr('And(x,Diamond(Not(x))) = Bot')
prover = Prover9Command(goal, assumptions = epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

#Check that "p or it must be that not p" is derivable
goal = read_expr('Or(x,Box(Not(x))) = Top')
prover = Prover9Command(goal, assumptions = epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42567 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 9.
% Level of proof is 3.
% Maximum clause weight is 9.000.
% Given clauses 24.


5 And(x,Diamond(Not(x))) = Bot.  [goal].
20 Not(Not(x)) = x.  [assumption].
30 Diamond(x) = Not(Box(Not(x))).  [assumption].
33 And(Not(x),Diamond(x)) = Bot.  [assumption].
34 And(Not(x),Not(Box(Not(x)))) = Bot.  [copy(33),rewrite([30(2)])].
35 And(c1,Diamond(Not(c1))) != Bot.  [deny(5)].
36 And(c1,Not(Box(c1))) != Bot.  [copy(35),rewrite([30(4),20(4

In [17]:
#Not only "p but might not p" but also "p but might might not p" is inconsisent 
goal = read_expr('And(x,Diamond(Diamond(Not(x))))=Bot')
prover = Prover9Command(goal, assumptions = epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42571 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.05 (+ 0.00) seconds.
% Length of proof is 31.
% Level of proof is 7.
% Maximum clause weight is 13.000.
% Given clauses 186.


5 And(x,Diamond(Diamond(Not(x)))) = Bot.  [goal].
8 Or(x,y) = Or(y,x).  [assumption].
9 And(x,y) = And(y,x).  [assumption].
12 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
13 And(And(x,y),z) = And(x,And(y,z)).  [copy(12),flip(a)].
14 Or(x,And(x,y)) = x.  [assumption].
16 Or(x,Bot) = x.  [assumption].
17 And(x,Top) = x.  [assumption].
18 Or(x,

In [18]:
#Check that Diamond p does not entail p
goal = read_expr('Diamond(x) = And(Diamond(x),x)')
mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 4.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 2.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 3.
Diamond(5) = 5.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 2.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 4.
And(3,5) = 0.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 4.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 2.
And(5,3) = 0.
And(5,4) = 0.
And(5,5) = 5.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(0,4) = 4.
Or(0,5) = 5.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(1,4) = 1.
Or(1,5) = 1.
Or(2,0) = 2

In [19]:
#Check that Diamond Diamond p does not entail Diamond p
goal = read_expr('Diamond(Diamond(x)) = Diamond(x)')
mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 8

Bot = 0.

Top = 1.

c1 = 4.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 6.
Box(6) = 2.
Box(7) = 4.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 7.
Diamond(5) = 5.
Diamond(6) = 5.
Diamond(7) = 3.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 2.
And(2,6) = 2.
And(2,7) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 4.
And(3,5) = 0.
And(3,6) = 0.
And(3,7) = 7.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 4.
And(4,4) = 4.
And(4,5) = 0.
And(4,6) = 0.
And(4,7) = 4.
And(5,0) = 0.
And(5,1) = 5.
And(

In [20]:
#Although "p and might not p" is inconsistent, "might p and must might not p" is consistent
goal = read_expr('And(Diamond(x),Box(Diamond(Not(x)))) = Bot')
mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 10

Bot = 0.

Top = 1.

c1 = 4.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 3.
Box(5) = 7.
Box(6) = 6.
Box(7) = 7.
Box(8) = 8.
Box(9) = 9.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 6.
Diamond(5) = 2.
Diamond(6) = 6.
Diamond(7) = 7.
Diamond(8) = 8.
Diamond(9) = 9.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 9.
Not(9) = 8.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 5.
And(2,6) = 8.
And(2,7) = 7.
And(2,8) = 8.
And(2,9) = 7.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 3.
And(3,5) = 0.
And(3,6

## 7. Conditional axioms<a id='7'></a>

In [21]:
if_and = read_expr('If(x,And(y,z)) = And(If(x,y),If(x,z))')
if_top = read_expr('If(x,Top) = Top')

ident = read_expr('If(x,x) = Top') 

simp_mp = read_expr('B(y) -> And(If(x,y),x) = And(And(If(x,y),x),y)')
simp_cs = read_expr('B(y) -> And(x,y) = And(And(x,y),If(x,y))')
simp_mt = read_expr('B(y) -> And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x))')

mod_mp = read_expr('And(Box(x),If(x,y))=And(And(Box(x),If(x,y)),y)')
mod_cs = read_expr('And(Box(x),y)=And(And(Box(x),y),If(x,y))')
mod_mt = read_expr('And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)), Not(Box(x)))')

must_intro = read_expr('x = And(x,y) -> If(x,Box(y)) = Top')

simp_must_import = read_expr('B(y) -> Box(If(x,y)) = And(Box(If(x,y)),If(x,Box(y)))')
safe_must_export = read_expr('B(x) -> If(x,Box(y)) = And(If(x,Box(y)),Box(If(x,y)))')

must_preserve = read_expr('And(Diamond(And(x,y)),Box(y)) = And(And(Diamond(And(x,y)),Box(y)),If(x,Box(y)))')

flat = read_expr('If(x,If(And(x,y),z)) = If(And(x,y),z)')
weak_boethius = read_expr('And(Diamond(x),If(x,y)) = And(And(Diamond(x),If(x,y)),Not(If(x,Not(y))))')
must_if_combo = read_expr('If(x,y) = And(If(x,y),Or(Not(x),And(Box(x),If(x,y))))')

safe_ni = read_expr('B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y)))')
safe_cem_plus = read_expr('B(x) -> If(x,Or(y,z)) = And(If(x,Or(y,z)),Or(If(x,y),If(x,z)))')


cond_ax = [if_and, if_top, ident,
           simp_mp, simp_cs, simp_mt, mod_mp, mod_cs, mod_mt,
           must_intro, simp_must_import, safe_must_export, must_preserve,
           flat, weak_boethius, must_if_combo,
           safe_ni, safe_cem_plus]

cond_modal_ortho_boolean_lattice = modal_ortho_boolean_lattice + cond_ax
cond_epistemic_ortho_boolean_lattice = epistemic_ortho_boolean_lattice + cond_ax

#Here we collect the axioms that don't involve modalities:
cond_ortho_boolean_lattice = ortho_boolean_lattice + [if_and, if_top, ident, simp_mp, simp_cs, simp_mt, flat, safe_ni, safe_cem_plus]

In [22]:
#Check that "If Diamond p, then p" is not valid
goal = read_expr('If(Diamond(x),x) = Top')
mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 4.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 2.
Diamond(5) = 5.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 4.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 3.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 4.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 3.
And(5,4) = 0.
And(5,5) = 5.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(0,4) = 1.
If(0,5) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(1,4) = 4.
If(1,5) = 5.
If(2,0) = 0

## 8. Independence of axioms<a id='8'></a>

Not all of the axioms in cond_ax are independent relative to cond_modal_ortho_boolean_lattice. 

In [23]:
#if_top is derivable from other axioms in cond_modal_ortho_boolean_lattice
goal = if_top
prover = Prover9Command(goal, assumptions = [ax for ax in cond_modal_ortho_boolean_lattice if not (ax == if_top)])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42581 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 11.000.
% Given clauses 31.


8 x = And(x,y) -> If(x,Box(y)) = Top.  [assumption].
13 If(x,Top) = Top.  [goal].
25 And(x,Top) = x.  [assumption].
37 Box(Top) = Top.  [assumption].
53 And(x,y) != x | If(x,Box(y)) = Top.  [clausify(8)].
69 If(c1,Top) != Top.  [deny(13)].
144 If(x,Top) = Top.  [hyper(53,a,25,a),rewrite([37(2)])].
145 $F.  [resolve(144,a,69,a)].

===================

In [24]:
#simp_mp is derivable from other axioms in cond_modal_ortho_boolean_lattice
goal = simp_mp
prover = Prover9Command(goal, assumptions = [ax for ax in cond_modal_ortho_boolean_lattice], timeout=100000)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42583 was started by halcrow on iMac.local,
Tue Feb 22 13:55:04 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 10.
% Level of proof is 4.
% Maximum clause weight is 15.000.
% Given clauses 40.


5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
13 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [goal].
17 And(x,y) = And(y,x).  [assumption].
42 -B(x) | And(If(y,x),y) = And(And(If(y,x),y),x).  [clausify(5)].
43 -B(x) | And(x,And(y,If(y,x))) = And(y,If(y,x)).  [copy(42),rewrite([17(3),17(5),17(6)]),flip(b)].
70 B(c1).  [deny(1

In [25]:
#ident is derivable from other axioms in cond_epistemic_ortho_boolean lattice
#but not cond_modal_ortho_boolean lattice
goal = ident
prover = Prover9Command(goal, assumptions = [ax for ax in cond_epistemic_ortho_boolean_lattice if not (ax == ident)])
prover.prove()
print(prover.proof())

mb = MaceCommand(goal, assumptions = [ax for ax in cond_modal_ortho_boolean_lattice if not (ax == ident)])

mb.build_model()
print(mb.model(format='cooked'))

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42585 was started by halcrow on iMac.local,
Tue Feb 22 13:55:05 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.03 (+ 0.00) seconds.
% Length of proof is 21.
% Level of proof is 6.
% Maximum clause weight is 13.000.
% Given clauses 150.


8 x = And(x,y) -> If(x,Box(y)) = Top.  [assumption].
13 If(x,x) = Top.  [goal].
15 And(x,x) = x.  [assumption].
16 Or(x,y) = Or(y,x).  [assumption].
17 And(x,y) = And(y,x).  [assumption].
22 Or(x,And(x,y)) = x.  [assumption].
23 And(x,Or(x,y)) = x.  [assumption].
25 And(x,Top) = x.  [assumption].
39 Box(x) = And(Box(x),x).  [assumption].
40 And(x,

Mace4 and Prover9 are unable to settle the independence or non-independence of a number of axioms, but Mace4 can show the independence of some.

In [26]:
for ax in cond_ax:
    if not (ax == if_top or ax == simp_mp or ax == simp_mt or
            ax == weak_boethius or ax == must_if_combo or
            ax == safe_ni or ax == safe_cem_plus):
        print(f"Is {ax} independent of the other axioms?")
        mace = Mace()
        print(mace.build_model(ax, assumptions = [axiom for axiom in cond_modal_ortho_boolean_lattice if not axiom == ax]))
        print("\n")

Is (If(x,And(y,z)) = And(If(x,y),If(x,z))) independent of the other axioms?
True


Is (If(x,x) = Top) independent of the other axioms?
True


Is (B(y) -> (And(x,y) = And(And(x,y),If(x,y)))) independent of the other axioms?
True


Is (And(Box(x),If(x,y)) = And(And(Box(x),If(x,y)),y)) independent of the other axioms?
True


Is (And(Box(x),y) = And(And(Box(x),y),If(x,y))) independent of the other axioms?
True


Is (And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(Box(x)))) independent of the other axioms?
True


Is ((x = And(x,y)) -> (If(x,Box(y)) = Top)) independent of the other axioms?
True


Is (B(y) -> (Box(If(x,y)) = And(Box(If(x,y)),If(x,Box(y))))) independent of the other axioms?
True


Is (B(x) -> (If(x,Box(y)) = And(If(x,Box(y)),Box(If(x,y))))) independent of the other axioms?
True


Is (And(Diamond(And(x,y)),Box(y)) = And(And(Diamond(And(x,y)),Box(y)),If(x,Box(y)))) independent of the other axioms?
True


Is (If(x,If(And(x,y),z)) = If(And(x,y),z)) independent of the other axiom

Next we verify that the non-simple versions of the simple conditional axioms are not derivable.

In [27]:
mp = read_expr('And(If(x,y),x) = And(And(If(x,y),x),y)')
cs = read_expr('And(x,y) = And(And(x,y),If(x,y))')
mt = read_expr('And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x))')
caut_trans = read_expr('And(If(x,y),If(And(x,y),z)) = And(And(If(x,y),If(And(x,y),z)),If(x,z))')
caut_mon = read_expr('And(If(x,y),If(x,z)) = And(And(If(x,y),If(x,z)),If(And(x,y),z))')
must_import = read_expr('Box(If(x,y)) = And(Box(If(x,y)),If(x,Box(y)))')
must_export = read_expr('If(x,Box(y)) = And(If(x,Box(y)),Box(If(x,y)))')
ni = read_expr('Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y)))')
cem_plus = read_expr('If(x,Or(y,z)) = And(If(x,Or(y,z)),Or(If(x,y),If(x,z)))')

non_simp_ax = [mp, cs, mt, caut_trans, caut_mon, must_import, must_export, ni, cem_plus]

In [28]:
for ax in non_simp_ax:
    print(f"Is {ax} a non-theorem?")
    mace = Mace()
    print(mace.build_model(ax, assumptions = cond_epistemic_ortho_boolean_lattice))
    print("\n")

Is (And(If(x,y),x) = And(And(If(x,y),x),y)) a non-theorem?
True


Is (And(x,y) = And(And(x,y),If(x,y))) a non-theorem?
True


Is (And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x))) a non-theorem?
True


Is (And(If(x,y),If(And(x,y),z)) = And(And(If(x,y),If(And(x,y),z)),If(x,z))) a non-theorem?
True


Is (And(If(x,y),If(x,z)) = And(And(If(x,y),If(x,z)),If(And(x,y),z))) a non-theorem?
True


Is (Box(If(x,y)) = And(Box(If(x,y)),If(x,Box(y)))) a non-theorem?
True


Is (If(x,Box(y)) = And(If(x,Box(y)),Box(If(x,y)))) a non-theorem?
True


Is (Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y)))) a non-theorem?
True


Is (If(x,Or(y,z)) = And(If(x,Or(y,z)),Or(If(x,y),If(x,z)))) a non-theorem?
True




## 9. The conditional epistemic ortholattice from Figure 12<a id='9'></a>

Below we code up the conditional epistemic ortholattice implied by Figure 12. We name the elements of the lattice from Figure 11 as $a$, $b$, $c$, $d$, $e$, $f$, $g$, $h$, $i$, and $j$ starting from the top of the Hasse diagram and working down, moving from left to right at each level of the Hasse diagram. For example, the node labelleled $\Box p\vee\Box\neg p$ in Figure 11 is node $c$, the node labelled $\neg p$ is node $f$, etc.

We then check that the specification of the lattice is consistent with cond_epistemic_ortho_boolean_lattice, i.e., the specification together with our axioms does not imply the contradiction $\neg(x=x)$. Since the specification completely describes the lattice, this shows that the conditional epistemic ortholattice implied by Figure 12 obeys all the axioms.

In [29]:
a_distinct = read_expr("-(a=b) & -(a=c) & -(a=d) & -(a=e) & -(a=f) & -(a=g) & -(a=h) & -(a=i) & -(a=j)")
b_distinct = read_expr("-(b=c) & -(b=d) & -(b=e) & -(b=f) & -(b=g) & -(b=h) & -(b=i) & -(b=j)")
c_distinct = read_expr("-(c=d) & -(c=e) & -(c=f) & -(c=g) & -(c=h) & -(c=i) & -(c=j)")
d_distinct = read_expr("-(d=e) & -(d=f) & -(d=g) & -(d=h) & -(d=i) & -(d=j)")
e_distinct = read_expr("-(e=f) & -(e=g) & -(e=h) & -(e=i) & -(e=j)")
f_distinct = read_expr("-(f=g) & -(f=h) & -(f=i) & -(f=j)")
g_distinct = read_expr("-(g=h) & -(g=i) & -(g=j)")
h_distinct = read_expr("-(h=i) & -(h=j)")
i_distinct = read_expr("-(i=j)")

distinct = [a_distinct, b_distinct, c_distinct, d_distinct, e_distinct, f_distinct, g_distinct, h_distinct, i_distinct]

elements = read_expr("all x.(x=a | x=b | x=c | x=d | x=e | x=f | x=g | x=h | x=i | x=j)")

a_top = read_expr("Top = a")
j_bot = read_expr("Bot = j")

bi_negs = read_expr("b = Not(i)")
ch_negs = read_expr("c = Not(h)")
dg_negs = read_expr("d = Not(g)")
ef_negs = read_expr("e = Not(f)")

e_under_b = read_expr("And(e,b)=e")
g_under_e = read_expr("And(g,e)=g")

c_join_of_gi = read_expr("c=Or(g,i)")
h_meet_of_bd = read_expr("h=And(b,d)")
g_meet_of_bc = read_expr("g=And(b,c)")
i_meet_of_cd = read_expr("i=And(c,d)")

box_op = read_expr("Box(a)=a & Box(b)=b & Box(c)=c & Box(d)=d & Box(e)=g & Box(f)=i & Box(g)=g & Box(h)=h & Box(i)=i & Box(j)=j")

boolean_sub = read_expr("B(a) & B(e) & B(f) & B(j)")

a_to = read_expr("If(a,a)=a & If(a,b)=b & If(a,c)=c & If(a,d)=d & If(a,e)=e & If(a,f)=f & If(a,g)=g & If(a,h)=h & If(a,i)=i & If(a,j)=j")
b_to = read_expr("If(b,a)=a & If(b,b)=a & If(b,c)=g & If(b,d)=h & If(b,e)=e & If(b,f)=j & If(b,g)=g & If(b,h)=h & If(b,i)=j & If(b,j)=j")
c_to = read_expr("If(c,a)=a & If(c,b)=e & If(c,c)=a & If(c,d)=f & If(c,e)=e & If(c,f)=f & If(c,g)=e & If(c,h)=j & If(c,i)=f & If(c,j)=j")
d_to = read_expr("If(d,a)=a & If(d,b)=h & If(d,c)=i & If(d,d)=a & If(d,e)=j & If(d,f)=f & If(d,g)=j & If(d,h)=h & If(d,i)=i & If(d,j)=j")
e_to = read_expr("If(e,a)=a & If(e,b)=a & If(e,c)=a & If(e,d)=j & If(e,e)=a & If(e,f)=j & If(e,g)=a & If(e,h)=j & If(e,i)=j & If(e,j)=j")
f_to = read_expr("If(f,a)=a & If(f,b)=j & If(f,c)=a & If(f,d)=a & If(f,e)=j & If(f,f)=a & If(f,g)=j & If(f,h)=j & If(f,i)=a & If(f,j)=j")
g_to = read_expr("If(g,a)=a & If(g,b)=a & If(g,c)=a & If(g,d)=j & If(g,e)=a & If(g,f)=j & If(g,g)=a & If(e,h)=j & If(e,i)=j & If(e,j)=j")
h_to = read_expr("If(h,a)=a & If(h,b)=a & If(h,c)=j & If(h,d)=a & If(h,e)=j & If(h,f)=j & If(h,g)=j & If(h,h)=a & If(h,i)=j & If(h,j)=j")
i_to = read_expr("If(i,a)=a & If(i,b)=j & If(i,c)=a & If(i,d)=a & If(i,e)=j & If(i,f)=a & If(i,g)=j & If(i,h)=j & If(i,i)=a & If(i,j)=j")
j_to = read_expr("If(j,a)=a & If(j,b)=a & If(j,c)=a & If(j,d)=a & If(j,e)=a & If(j,f)=a & If(j,g)=a & If(j,h)=a & If(j,i)=a & If(j,j)=a")

lattice_spec = distinct + [elements, a_top, j_bot, bi_negs, ch_negs, dg_negs, ef_negs, 
                           e_under_b, g_under_e, c_join_of_gi, g_meet_of_bc, 
                           h_meet_of_bd, i_meet_of_cd, box_op, boolean_sub,
                           a_to, b_to, c_to, d_to, e_to, f_to, g_to, h_to, i_to, j_to]

In [30]:
goal = read_expr("-(x=x)")
mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice + lattice_spec + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 10

Bot = 0.

Top = 1.

a = 1.

b = 2.

c = 3.

d = 4.

e = 5.

f = 6.

g = 7.

h = 8.

i = 9.

j = 0.

c1 = 0.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 7.
Box(6) = 9.
Box(7) = 7.
Box(8) = 8.
Box(9) = 9.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 4.
Diamond(5) = 2.
Diamond(6) = 4.
Diamond(7) = 7.
Diamond(8) = 8.
Diamond(9) = 9.

Not(0) = 1.
Not(1) = 0.
Not(2) = 9.
Not(3) = 8.
Not(4) = 7.
Not(5) = 6.
Not(6) = 5.
Not(7) = 4.
Not(8) = 3.
Not(9) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 7.
And(2,4) = 8.
And(2,5) = 5.
And(2,6) = 0.
And(2,7) = 7.
And(2,8) = 8.
And(2,9) = 0.
And(3,0) = 

## 10. Example Proofs<a id='10'></a>

We show that the following principles are derivable:

$\varphi\to\psi\vdash\neg\varphi\vee\psi$ (If-to-Or);

$(\varphi\to\psi)\wedge \varphi\wedge\neg\psi\vdash\bot$ (If Contradiction);

if $\varphi\vdash\psi$, then $\psi\to\bot\vdash \varphi\to\bot$ (Falsum Reversal);

$\varphi\to\bot \vdash \neg\varphi$ (Ad Falsum);

$\varphi\vdash \top\to \varphi$ and $\top\to \varphi \vdash\varphi$ (Trivial Conditioning).

In [31]:
#Derivable principles
if_to_or = read_expr('If(x,y) = And(If(x,y),Or(Not(x),y))')
if_contra = read_expr('And(If(x,y),And(x,Not(y))) = Bot')
falsum_rev = read_expr('x = And(x,y) -> If(y,Bot) = And(If(y,Bot),If(x,Bot))')
ad_falsum = read_expr('If(x,Bot) = And(If(x,Bot),Not(x))')
cond_triv = read_expr('x = And(x,If(Top,x))')

In [32]:
goal = if_to_or
prover = Prover9Command(goal, assumptions = modal_ortho_boolean_lattice + [mod_mp, must_if_combo])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42623 was started by halcrow on iMac.local,
Tue Feb 22 13:55:33 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 18.07 (+ 0.61) seconds.
% Length of proof is 24.
% Level of proof is 7.
% Maximum clause weight is 17.000.
% Given clauses 3148.


5 If(x,y) = And(If(x,y),Or(Not(x),y)).  [goal].
8 Or(x,y) = Or(y,x).  [assumption].
9 And(x,y) = And(y,x).  [assumption].
10 Or(x,Or(y,z)) = Or(Or(x,y),z).  [assumption].
11 Or(Or(x,y),z) = Or(x,Or(y,z)).  [copy(10),flip(a)].
14 Or(x,And(x,y)) = x.  [assumption].
15 And(x,Or(x,y)) = x.  [assumption].
31 And(Box(x),If(x,y)) = And(And(Box(x),If(x,

In [33]:
goal = if_contra
prover = Prover9Command(goal, assumptions = modal_ortho_boolean_lattice + [mod_mp, must_if_combo])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42628 was started by halcrow on iMac.local,
Tue Feb 22 13:55:52 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 4.92 (+ 0.16) seconds.
% Length of proof is 41.
% Level of proof is 9.
% Maximum clause weight is 20.000.
% Given clauses 1688.


5 And(If(x,y),And(x,Not(y))) = Bot.  [goal].
6 Or(x,x) = x.  [assumption].
8 Or(x,y) = Or(y,x).  [assumption].
9 And(x,y) = And(y,x).  [assumption].
10 Or(x,Or(y,z)) = Or(Or(x,y),z).  [assumption].
11 Or(Or(x,y),z) = Or(x,Or(y,z)).  [copy(10),flip(a)].
12 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
13 And(And(x,y),z) = And(x,And(y,z)).  [co

In [34]:
goal = falsum_rev
prover = Prover9Command(goal, assumptions = cond_modal_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42631 was started by halcrow on iMac.local,
Tue Feb 22 13:55:57 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.31 (+ 0.01) seconds.
% Length of proof is 43.
% Level of proof is 7.
% Maximum clause weight is 19.000.
% Given clauses 484.


1 B(x) -> B(Not(x)).  [assumption].
4 B(x) & B(y) & B(z) -> And(x,Or(y,z)) = Or(And(x,y),And(x,z)).  [assumption].
13 x = And(x,y) -> If(y,Bot) = And(If(y,Bot),If(x,Bot)).  [goal].
15 And(x,x) = x.  [assumption].
16 Or(x,y) = Or(y,x).  [assumption].
17 And(x,y) = And(y,x).  [assumption].
20 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
21 And(

In [35]:
goal = ad_falsum
prover = Prover9Command(goal, assumptions = cond_modal_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42633 was started by halcrow on iMac.local,
Tue Feb 22 13:55:57 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 27.
% Level of proof is 5.
% Maximum clause weight is 19.000.
% Given clauses 42.


1 B(x) -> B(Not(x)).  [assumption].
4 B(x) & B(y) & B(z) -> And(x,Or(y,z)) = Or(And(x,y),And(x,z)).  [assumption].
7 B(x) -> And(If(y,x),Not(x)) = And(And(If(y,x),Not(x)),Not(y)).  [assumption].
13 If(x,Bot) = And(If(x,Bot),Not(x)).  [goal].
15 And(x,x) = x.  [assumption].
16 Or(x,y) = Or(y,x).  [assumption].
17 And(x,y) = And(y,x).  [assumption].


In [36]:
goal = cond_triv
prover = Prover9Command(goal, assumptions = cond_modal_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 42635 was started by halcrow on iMac.local,
Tue Feb 22 13:55:57 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 16.
% Level of proof is 4.
% Maximum clause weight is 15.000.
% Given clauses 30.


13 x = And(x,If(Top,x)).  [goal].
17 And(x,y) = And(y,x).  [assumption].
20 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
21 And(And(x,y),z) = And(x,And(y,z)).  [copy(20),flip(a)].
25 And(x,Top) = x.  [assumption].
37 Box(Top) = Top.  [assumption].
48 And(Box(x),If(x,y)) = And(And(Box(x),If(x,y)),y).  [assumption].
49 And(x,And(Box(y),If(y,x)))

## 11. Avoiding collapse<a id='11'></a>

Next we show that $\alpha\to\beta \not\equiv \neg\alpha\vee\beta $ (No Simple Collapse).

In [37]:
goal = read_expr('B(x) & B(y) -> If(x,y) = Or(Not(x),y)')
mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

c1 = 2.

c2 = 0.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.
If(2,3) = 0.
If(3,0) = 0.
If(3,1) = 1.
If(3,2) = 0.
If(3,3) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.

f1(0,0) = 0.
f1(0,1) = 2.
f1(0,2) = 0.
f1(0,3) = 0.
f1(1,0) = 0.
f1(1,1) = 0.
f1(1,2) = 0.
f1(1,3) =

## 12. Modalized Import-Export<a id='12'></a>

Consider, $(\varphi\to \Diamond(\varphi\wedge\psi))\wedge (\varphi \to (\psi\to\chi)) \equiv (\varphi\to \Diamond(\varphi\wedge\psi)) \wedge ((\varphi\wedge\psi)\to\chi)$ (Modalized Import-Export), which is proved in the paper. Unfortunately, Prover9 does not find the proof before timing out.

In [38]:
mie = read_expr('And(If(x,Diamond(And(x,y))),If(x,If(y,z))) = And(If(x,Diamond(And(x,y))),If(And(x,y),z))')

In [39]:
#A simple version of mie with x, y, and z non-epistemic
simp_mie = read_expr('((B(x) & B(z)) & B(z)) -> And(If(x,Diamond(And(x,y))),If(x,If(y,z))) = And(If(x,Diamond(And(x,y))),If(And(x,y),z))')

In [40]:
#goal = mie
#prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice, timeout=10000)
#prover.prove()
#print(prover.proof())

In [41]:
#goal = mie
#mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
#mb.build_model()
#print(mb.model(format='cooked'))

Mace4 is able to show that without Must Preservation, Modalized Import-Export is not derivable.

In [42]:
goal = mie
mb = MaceCommand(goal, assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                      if not axiom == must_preserve])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 13

% Interpretation of size 10

Bot = 0.

Top = 1.

c1 = 2.

c2 = 4.

c3 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 9.
Box(6) = 6.
Box(7) = 3.
Box(8) = 8.
Box(9) = 9.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 8.
Diamond(5) = 5.
Diamond(6) = 2.
Diamond(7) = 7.
Diamond(8) = 8.
Diamond(9) = 9.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 9.
Not(9) = 8.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 6.
And(2,5) = 0.
And(2,6) = 6.
And(2,7) = 0.
And(2,8) = 2.
And(2,9) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
An

In [43]:
#The non-epistemic version of mie is also not derivable without Must Preservation
goal = simp_mie
mb = MaceCommand(goal, assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                      if not axiom == must_preserve])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 26

% Interpretation of size 12

Bot = 0.

Top = 1.

c1 = 2.

c2 = 2.

c3 = 4.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 8.
Box(6) = 6.
Box(7) = 9.
Box(8) = 8.
Box(9) = 9.
Box(10) = 10.
Box(11) = 11.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 10.
Diamond(5) = 5.
Diamond(6) = 11.
Diamond(7) = 7.
Diamond(8) = 8.
Diamond(9) = 9.
Diamond(10) = 10.
Diamond(11) = 11.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 10.
Not(9) = 11.
Not(10) = 8.
Not(11) = 9.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(0,10) = 0.
And(0,11) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(1,10) = 10.
And(1,11) = 11.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.


## 13. Qualified Collapse<a id='13'></a>

In Proposition 6.20 in the paper, we consider Qualified Collapse: $\psi\wedge (\psi\to \Diamond (\varphi\wedge\psi))\vdash \varphi\to\psi$.

In [44]:
q_collapse = read_expr('And(y,If(y,Diamond(And(x,y)))) = And(And(y,If(y,Diamond(And(x,y)))), If(x,y))')

In [45]:
#A simple version of q_collapse with x,y non-epistemic
simp_q_collapse = read_expr('(B(x) & B(y)) -> And(y,If(y,Diamond(And(x,y)))) = And(And(y,If(y,Diamond(And(x,y)))), If(x,y))')

In [46]:
#If we drop distributivity from the assumptions of Proposition 6.20, 
#then Mace4 finds a counterexample to even simple q_collapse,
#which satisfies not only modal_ortho_boolean_lattice but even epistemic_ortho_boolean_lattice

goal = simp_q_collapse
mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice + 
                                     [if_and,if_top,ident,if_to_or,mie])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 1.

c2 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 2.
Diamond(5) = 5.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 4.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 3.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 4.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 3.
And(5,4) = 0.
And(5,5) = 5.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(0,4) = 1.
If(0,5) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 4.
If(1,3) = 3.
If(1,4) = 4.
If(1,5) = 3.
If

In [47]:
#In fact, Mace4 finds a model falsifying q_collapse while satisfying 
#all of cond_epistemic_ortho_boolean_lattice.

goal = q_collapse
mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 8

Bot = 0.

Top = 1.

c1 = 2.

c2 = 6.

Box(0) = 0.
Box(1) = 1.
Box(2) = 5.
Box(3) = 3.
Box(4) = 4.
Box(5) = 5.
Box(6) = 6.
Box(7) = 7.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 4.
Diamond(4) = 4.
Diamond(5) = 5.
Diamond(6) = 6.
Diamond(7) = 7.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 5.
And(2,6) = 5.
And(2,7) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 3.
And(3,5) = 0.
And(3,6) = 0.
And(3,7) = 0.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 3.
And(4,4) = 4.
And(4,5) = 0.
And(4,6) = 0.
And(4,7) = 7.
And(5,0) = 0.
And(5,1) 

In [48]:
#It's also noteworthy that the assumptions of Proposition 6.20 
#do not entail our episteme axiom

goal = episteme
mb = MaceCommand(goal, assumptions = modal_ortho_boolean_lattice + 
                                     [if_and,if_top,ident,if_to_or,mie] +
                                     [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

c1 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 0.
Box(3) = 0.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 1.
Diamond(3) = 1.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.
If(2,3) = 0.
If(3,0) = 0.
If(3,1) = 1.
If(3,2) = 0.
If(3,3) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.

f1(0,0) = 0.
f1(0,1) = 2.
f1(0,2) = 0.
f1(0,3) = 0.
f1(1,0) = 0.
f1(1,1) = 0.
f1(1,2) = 0.
f1(1,3) = 0.
f1(2,

Unfortunately, to date Mace4 has not found a model falsifying simp_q_collapse while satisfying all of cond_epistemic_ortho_boolean_lattice, while Prover9 has not found a proof of simp_q_collapse from cond_epistemic_ortho_boolean_lattice.

In [49]:
#goal = simp_q_collapse
#mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
#mb.build_model()
#print(mb.model(format='cooked'))

In [50]:
#goal = simp_q_collapse
#prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice, timeout=1000000)
#prover.prove()
#print(prover.proof())

However, we have some partial results.

First, Mace4 can find a model falsifying simp_q_collapse while satisfying all of cond_epistemic_ortho_boolean_lattice except for mod_cs, simp_cs, and safe_ni.

In [51]:
goal = simp_q_collapse
mb = MaceCommand(goal, assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                      if not (axiom == mod_cs or axiom == simp_cs or axiom == safe_ni)])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 1.

c2 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 4.
Box(3) = 3.
Box(4) = 4.
Box(5) = 5.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 5.
Diamond(4) = 4.
Diamond(5) = 5.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 4.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 3.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 4.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 3.
And(5,4) = 0.
And(5,5) = 5.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(0,4) = 1.
If(0,5) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 4.
If(1,3) = 3.
If(1,4) = 4.
If(1,5) = 5.
If

Second, Mace4 can find a model falsifying simp_q_collapse while satisfying all of cond_epistemic_ortho_boolean_lattice except for flat (although a restricted version of flat works), safe_must_export, and must_preserve.

In [52]:
simp_flat = read_expr('B(x) -> If(x,If(And(x,y),z)) = If(And(x,y),z)')

goal = simp_q_collapse
mb = MaceCommand(goal, 
                 assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                if not (axiom == flat 
                                        or axiom == safe_must_export 
                                        or axiom == must_preserve)] 
                                + [simp_flat])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 1

% Interpretation of size 12

Bot = 0.

Top = 1.

c1 = 2.

c2 = 6.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 5.
Box(4) = 4.
Box(5) = 5.
Box(6) = 6.
Box(7) = 7.
Box(8) = 8.
Box(9) = 11.
Box(10) = 10.
Box(11) = 11.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 4.
Diamond(3) = 3.
Diamond(4) = 4.
Diamond(5) = 5.
Diamond(6) = 6.
Diamond(7) = 7.
Diamond(8) = 10.
Diamond(9) = 9.
Diamond(10) = 10.
Diamond(11) = 11.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 9.
Not(9) = 8.
Not(10) = 11.
Not(11) = 10.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(0,10) = 0.
And(0,11) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(1,10) = 10.
And(1,11) = 11.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) =

Third, we can find a counterexample to the last distributivity step in the proof of Proposition 6.24, namely $\psi\wedge (\neg\psi\vee (\varphi\to\psi))\vdash \psi\wedge (\varphi\to\psi)$, relative to all our examples except for safe_ni and safe_cem_plus.

In [53]:
goal = read_expr("(B(x) & B(y)) -> And(y,Or(Not(y),If(x,y)))=And(y,If(x,y))")

mb = MaceCommand(goal, assumptions = [ax for ax in cond_epistemic_ortho_boolean_lattice 
                                      if not(ax == safe_ni or ax ==safe_cem_plus)])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 10

Bot = 0.

Top = 1.

c1 = 2.

c2 = 4.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 5.
Box(6) = 6.
Box(7) = 7.
Box(8) = 8.
Box(9) = 9.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 4.
Diamond(5) = 5.
Diamond(6) = 6.
Diamond(7) = 7.
Diamond(8) = 8.
Diamond(9) = 9.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 9.
Not(9) = 8.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 6.
And(2,5) = 5.
And(2,6) = 6.
And(2,7) = 5.
And(2,8) = 6.
And(2,9) = 5.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 3.
And(3,5) = 0

Fourth, we can find a counterexample to the related principle $\psi\wedge(\psi\to(\varphi\to\psi))\vdash \varphi\to\psi$ for $\varphi,\psi$ Boolean relative to all our axioms except for simp_cs or mod_cs (though a very simple version of cs is okay).

In [54]:
very_simp_cs = read_expr("(B(x) & B(y)) -> And(x,y) = And(And(x,y),If(x,y))")

goal = read_expr("(B(x) & B(y)) -> And(x,If(x,If(y,x))) = And(And(x,If(x,If(y,x))),If(y,x)) ")

mb = MaceCommand(goal, assumptions = [ax for ax in cond_epistemic_ortho_boolean_lattice 
                                      if not(ax == simp_cs or ax ==mod_cs)] + [very_simp_cs])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 10

Bot = 0.

Top = 1.

c1 = 2.

c2 = 3.

Box(0) = 0.
Box(1) = 1.
Box(2) = 7.
Box(3) = 5.
Box(4) = 4.
Box(5) = 5.
Box(6) = 6.
Box(7) = 7.
Box(8) = 8.
Box(9) = 9.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 4.
Diamond(3) = 6.
Diamond(4) = 4.
Diamond(5) = 5.
Diamond(6) = 6.
Diamond(7) = 7.
Diamond(8) = 8.
Diamond(9) = 9.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 9.
Not(9) = 8.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 2.
And(2,5) = 0.
And(2,6) = 0.
And(2,7) = 7.
And(2,8) = 0.
And(2,9) = 7.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 5

## 14. Provable principles for which Prover9 does not find a proof <a id='14'></a>

In the paper, we prove Modalized Cautious Transitivity and Modalized Cautious Monotonicity (resp. Simple Cautious Transitivity and Simple Cautious Monotonicity). Unfortunately Prover9/Mace4 does not find a proof/counterexample.

In [55]:
mod_caut_trans = read_expr('And(If(x,Box(y)),If(And(x,y),z)) = And(And(If(x,Box(y)),If(And(x,y),z)),If(x,z))')
simp_caut_trans = read_expr('B(z) -> And(If(x,y),If(And(x,y),z)) = And(And(If(x,y),If(And(x,y),z)),If(x,z))')

In [56]:
mod_caut_mon = read_expr('And(If(x,Box(y)),If(x,z)) = And(And(If(x,Box(y)),If(x,z)), If(And(x,y),z))')
simp_caut_mon = read_expr('B(z) -> And(If(x,y),If(x,z)) = And(And(If(x,y),If(x,z)),If(And(x,y),z))')

In [57]:
#goal = mod_caut_trans
#prover = Prover9Command(goal, assumptions = modal_ortho_boolean_lattice 
#                        + [if_and,if_top,mod_mp,must_intro,flat],timeout=1000000)
#prover.prove()
#print(prover.proof())

In [58]:
#goal = mod_caut_trans
#mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice 
#                        + [if_and,if_top,mod_mp,must_intro,flat])
#mb.build_model()
#print(mb.model(format='cooked'))

In [59]:
#goal = mod_caut_mon
#prover = Prover9Command(goal, assumptions = modal_ortho_boolean_lattice 
#                        + [if_and,if_top,mod_cs,must_intro,flat],timeout=1000000)
#prover.prove()
#print(prover.proof())

In [60]:
#goal = mod_caut_mon
#mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice 
#                        + [if_and,if_top,mod_cs,must_intro,flat])
#mb.build_model()
#print(mb.model(format='cooked'))

Consider $(\varphi\to\psi)\wedge (\psi\to\bot)\vdash \varphi\to\bot$ (Conditional Modus Tollens), which is provable as follows:

1. $\psi\to\bot \vdash \psi\to\varphi$ and $\psi\to\bot\vdash\psi\to\bot$, so $\psi\to\bot \vdash (\psi\to\varphi)\wedge (\psi\to\bot)$
2. By Simple Cautious Monotonicity, $(\psi\to\varphi)\wedge (\psi\to\bot)\vdash (\psi\wedge\varphi)\to\bot\vdash (\varphi\wedge\psi)\to\bot$.
3. By 1 and 2, $\psi\to\bot \vdash (\varphi\wedge\psi)\to\bot$.
4. By Simple Cautious Transitivity, $(\varphi\to\psi)\wedge (\varphi\wedge\psi)\to\bot\vdash \varphi\to\bot$.
5. By 3 and 4, $(\varphi\to\psi)\wedge (\psi\to\bot)\vdash \varphi\to\bot$.

Unfortunately Prover9/Mace4 does not find a proof/counterexample.

In [61]:
cond_mt = read_expr('And(If(x,y),If(y,Bot)) = And(And(If(x,y),If(y,Bot)),If(x,Bot))')

In [62]:
#goal = cond_mt
#prover = Prover9Command(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#prover.prove()
#print(prover.proof())

In [63]:
#goal = cond_mt
#mb = MaceCommand(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#mb.build_model()
#print(mb.model(format='cooked'))

Consider, for non-epistemic $\beta$, $(\varphi\to\psi)\wedge (\psi\to\varphi)\wedge (\psi\to\beta)\vdash \varphi\to\beta$ (Simple Reciprocity), which is provable as follows:

1. By Simple Cautious Monotonicity,  $(\psi\to\varphi)\wedge (\psi\to\beta)\vdash (\psi\wedge\varphi)\to\beta\vdash (\varphi\wedge\psi)\to\beta$.

2. By Simple Cautious Transitivity, $(\varphi\to\psi)\wedge ((\varphi\wedge\psi)\to\beta)\vdash \varphi\to\beta$.

3. By 1 and 2, $(\varphi\to\psi)\wedge (\psi\to\varphi)\wedge (\psi\to\beta)\vdash \varphi\to\beta$.

Unfortunately Prover9/Mace4 does not find a proof/counterexample.

In [64]:
simp_rep = read_expr('B(z) -> And(And(If(x,y),If(y,x)),If(y,z)) = And(And(And(If(x,y),If(y,x)),If(y,z)),If(x,z))')

In [65]:
#goal = simp_rep
#prover = Prover9Command(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#prover.prove()
#print(prover.proof())

In [66]:
#goal = simp_rep
#mb = MaceCommand(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#mb.build_model()
#print(mb.model(format='cooked'))

## 15. A more economical axiomatization without $\vee$, $\bot$, or $\Diamond$ <a id='15'></a>

In [67]:
econ_contra = read_expr('And(x,Not(x))= Not(Top)')
econ_de_morgan = read_expr('And(x,Not(And(Not(x),Not(y)))) = x')

econ_ortho_lattice = [and_id, and_comm, and_assoc, top, econ_contra, invol, econ_de_morgan]

In [68]:
econ_B_dist = read_expr('((B(x) & B(y)) & B(z)) -> And(x,Not(And(Not(y),Not(z)))) = Not(And(Not(And(x,y)),Not(And(x,z))))')

econ_ortho_boolean_lattice = econ_ortho_lattice + [B_top, B_not, B_and, econ_B_dist]

In [69]:
econ_modal_ortho_boolean_lattice = econ_ortho_boolean_lattice + [box_and, box_top]

In [70]:
econ_episteme = read_expr('And(x,Not(Box(x))) = Not(Top)')
econ_epistemic_ortho_boolean_lattice = econ_modal_ortho_boolean_lattice + [factive, econ_episteme]

In [71]:
econ_must_preserve = read_expr('And(Not(Box(Not(And(x,y)))),Box(y)) = And(And(Not(Box(Not(And(x,y)))),Box(y)),If(x,Box(y)))')
econ_weak_boethius = read_expr('And(Not(Box(Not(x))),If(x,y)) = And(And(Not(Box(Not(x))),If(x,y)),Not(If(x,Not(y))))')
econ_must_if_combo = read_expr('If(x,y) = And(If(x,y),Not(And(x,Not(And(Box(x),If(x,y))))))')
econ_safe_cem_plus = read_expr('B(x) -> If(x,Not(And(Not(y),Not(z)))) = And(If(x,Not(And(Not(y),Not(z)))),Not(And(Not(If(x,y)),Not(If(x,z)))))')

econ_cond_ax = [if_and, if_top, ident,
               mod_mp, mod_cs, simp_cs, simp_mt,
               must_intro, simp_must_import, safe_must_export, econ_must_preserve,
               flat, econ_weak_boethius, econ_must_if_combo,
               safe_ni, econ_safe_cem_plus]

econ_cond_modal_ortho_boolean_lattice = econ_modal_ortho_boolean_lattice + econ_cond_ax
econ_cond_epistemic_ortho_boolean_lattice = econ_epistemic_ortho_boolean_lattice + econ_cond_ax